In [9]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
import os
from dotenv import load_dotenv

load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

# 문서 로드
loader = PyPDFLoader('../data/KCI_FI003153549.pdf')
documents = loader.load()

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
splitted_documents = text_splitter.split_documents(documents)

# 임베딩 모델 준비
embedding_model = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key=gemini_api_key,
)

# # FAISS 벡터스토어 생성 및 저장
# vectorstore = FAISS.from_documents(splitted_documents, embedding_model)
# vectorstore.save_local("faiss_index")

# # 벡터스토어 재로딩
# reloaded_store = FAISS.load_local(
#     "faiss_index",
#     embedding_model,
#     allow_dangerous_deserialization=True,
# )

# FAISS 벡터스토어가 존재하는 경우에는 덮어쓰기 하지 않고 로드
FAISS_INDEX_PATH = "faiss_index"

vectorstore = None

if os.path.exists(FAISS_INDEX_PATH):
    print(f"FAISS 인덱스 {FAISS_INDEX_PATH}를 로드합니다.")
    vectorstore = FAISS.load_local(
        FAISS_INDEX_PATH,
        embedding_model,
        allow_dangerous_deserialization=True,
    )
else:
    print(f"FAISS 인덱스 {FAISS_INDEX_PATH}가 없으므로 생성합니다.")
    # 문서 로드
    loader = PyPDFLoader('../data/KCI_FI003153549.pdf')
    documents = loader.load()

    # 문서 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
    )
    splitted_documents = text_splitter.split_documents(documents)

    # FAISS 벡터스토어 생성 및 저장
    vectorstore = FAISS.from_documents(splitted_documents, embedding_model)
    vectorstore.save_local(FAISS_INDEX_PATH)
    print(f"FAISS 인덱스를 {FAISS_INDEX_PATH}에 저장했습니다.")

FAISS 인덱스 faiss_index를 로드합니다.


In [10]:
vectorstore

In [11]:
# 예시 질의
query = "본 연구에서 Private LLM 구축을 위해 수집한 문서의 총 페이지 수와 문서 유형별 비율은 어떻게 되나요?"
# query = "Advance RAG 기법이 임상시험 데이터 분석에서 수행하는 주요 역할은 무엇인가요?"
# query = "본 연구에서 Private LLM 성능을 평가하기 위해 사용한 지표 3가지는 무엇인가요?"
# query = "국내에서 LLM을 임상시험에 적용한 대표적인 기관과 그 적용 사례를 2가지 이상 말해보세요."
# query = "ROUGE 평가에서 Private LLM과 ChatGPT의 Recall 값은 각각 얼마였나요?"

# results = vectorstore.similarity_search(query, k=3) # k는 유사도 검색에서 반환할 상위 문서 개수(top‑k)
results = vectorstore.similarity_search(query, k=5)

for idx, doc in enumerate(results, start=1):
    print(f"[결과 {idx}]\n" + doc.page_content[:300])
    print("---")

[결과 1]
의료기기 임상시험 분야의 도메인 특성에 맞게 튜닝하
기 위해 의료기기 임상시험 전문가로부터 총 158개의 문
서(총 11,954 페이지)를 수집하였다. 수집된 문서는 다음
과 같이 분류된다:
 규제 문서 (30%): FDA, EMA, PMDA 가이드라인, 
GCP 문서 등
 교육 자료 (20%): 임상시험 수행자 교육 매뉴얼, 온라
인 강의 자료 등
 프로토콜 및 보고서 (25%): 임상시험 프로토콜, CSR 
(Clinical Study Report) 템플릿 등
 의료기기 특화 문서 (15%): 의료기기 임상시험 계획
서,
---
[결과 2]
174   Journal of The Korea Society of Computer and Information 
문서(10%)로 구성되며, 각 분류는 도메인 전문가의 검토를 
통해 정확성과 신뢰성을 확보하였다. 특히, 주요 규제 기
관(FDA, EMA, PMDA) 가이드라인, GCP 문서, 환자 동의
서 템플릿 등은 모델이 국제 표준에 기반하여 학습할 수 
있도록 하였고, 교육 자료와 프로토콜은 실질적인 임상시
험 수행과 데이터 관리 작업에서 발생할 수 있는 질문들에 
대응할 수 있는 기초를 제공한다.
이 데이터셋은 다양한 문서 
---
[결과 3]
This study explores the improvement of work efficiency and expertise by applying Private LLM 
based on Large Language Model (LLM) to the field of clinical trials in medical devices. The Private 
LLM system provides sophisticated and accurate answers based on clinical data and shows its potential 
fo
---
[결과 4]
170   Journal of The Korea Society of Computer and Inform

In [12]:
query

'본 연구에서 Private LLM 구축을 위해 수집한 문서의 총 페이지 수와 문서 유형별 비율은 어떻게 되나요?'

#### LCEL 파이프라인은 컴포넌트 간의 명확한 흐름을 강조함

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

prompt = ChatPromptTemplate.from_template(
    '''다음 컨텍스트만 사용해 질문에 답하세요.
컨텍스트:{context}

질문: {question}
'''
)

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=gemini_api_key)
chain = prompt | llm

# 관련 문서를 사용한 답변
result = chain.invoke({'context': results, 'question': query})

print(result)

content='본 연구에서 Private LLM 구축을 위해 수집한 문서의 총 페이지 수는 **11,954 페이지**입니다.\n\n문서 유형별 비율은 다음과 같습니다:\n*   **규제 문서**: 30% (FDA, EMA, PMDA 가이드라인, GCP 문서 등)\n*   **교육 자료**: 20% (임상시험 수행자 교육 매뉴얼, 온라인 강의 자료 등)\n*   **프로토콜 및 보고서**: 25% (임상시험 프로토콜, CSR 템플릿 등)\n*   **의료기기 특화 문서**: 15% (의료기기 임상시험 계획서, 기술문서 등)\n*   **기타**: 10% (윤리위원회 관련 문서, 환자 동의서 템플릿 등)' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []} id='run--93c77224-1d19-48ea-89e8-06ced56f2c52-0' usage_metadata={'input_tokens': 2885, 'output_tokens': 726, 'total_tokens': 3611, 'input_token_details': {'cache_read': 2032}, 'output_token_details': {'reasoning': 543}}


In [16]:
print(result.content)

본 연구에서 Private LLM 구축을 위해 수집한 문서의 총 페이지 수는 **11,954 페이지**입니다.

문서 유형별 비율은 다음과 같습니다:
*   **규제 문서**: 30% (FDA, EMA, PMDA 가이드라인, GCP 문서 등)
*   **교육 자료**: 20% (임상시험 수행자 교육 매뉴얼, 온라인 강의 자료 등)
*   **프로토콜 및 보고서**: 25% (임상시험 프로토콜, CSR 템플릿 등)
*   **의료기기 특화 문서**: 15% (의료기기 임상시험 계획서, 기술문서 등)
*   **기타**: 10% (윤리위원회 관련 문서, 환자 동의서 템플릿 등)


#### @chain 데코레이터는 여러 작업을 하나의 함수로 묶어(여러 단계를 하나의 함수 안에 캡슐화) 코드를 구조화함

In [ ]:
from langchain_core.runnables import chain

@chain
def qa(input):
    # 관련 문서 검색
    docs = vectorstore.similarity_search(input, k=5)
    # 프롬프트 포매팅
    formatted = prompt.invoke({'context': docs, 'question': input})
    # 답변 생성
    answer = llm.invoke(formatted)
    return answer
    # return {"answer": answer, "docs": docs}

# 실행
result = qa.invoke(query)
print(result.content)

본 연구에서 Private LLM 구축을 위해 수집한 문서의 총 페이지 수는 **11,954 페이지**입니다.

문서 유형별 비율은 다음과 같습니다:
*   **규제 문서**: 30% (FDA, EMA, PMDA 가이드라인, GCP 문서 등)
*   **교육 자료**: 20% (임상시험 수행자 교육 매뉴얼, 온라인 강의 자료 등)
*   **프로토콜 및 보고서**: 25% (임상시험 프로토콜, CSR 템플릿 등)
*   **의료기기 특화 문서**: 15% (의료기기 임상시험 계획서, 기술문서 등)
*   **기타**: 10% (윤리위원회 관련 문서, 환자 동의서 템플릿 등)
